In [1]:
#importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pystan
import scipy.stats as sts
sns.set()

In [2]:

#importing the data and previewing
data = pd.read_csv('/Users/anirudhn/Desktop/Code/CS146_Assignments/LBA/Data.csv', na_values='n/a')
data.head(n=3)

,Timestamp,Email Address,Your name,Country where you (and the grocery store) are,"Currency (e.g. EUR, GBP, USD)",Grocery store name and street address (or URL if shopping online),Price perception of the store brand,Average rental price near the grocery store,Apple 1 quantity (kg),Apple 1 price,...,Eggs 2 quantity (count),Eggs 2 price,Eggs 3 quantity (count),Eggs 3 price,Chicken 1 quantity (kg),Chicken 1 price,Chicken 2 quantity (kg),Chicken 2 price,Chicken 3 quantity (kg),Chicken 3 price
0,10/20/2020 23:22:49,tiagoflora@minerva.kgi.edu,Tiago Flora,Germany,EUR,"EDEKA, Grunerstraße 20",Mid-range,EUR900,1.0,2.93,...,6.0,1.75,4.0,1.99,0.312,3.97,0.60,3.87,0.400,2.90
1,10/20/2020 23:32:18,tiagoflora@minerva.kgi.edu,Tiago Flora,Germany,EUR,"EDEKA Andreasstraße 59, 10243 Berlin",Mid-range,EUR850,1.0,1.95,...,6.0,2.73,10.0,1.63,0.394,5.02,0.43,7.17,0.311,9.12
2,10/23/2020 18:39:04,katja.dellalibera@minerva.kgi.edu,Katja Della Libera,Germany,EUR,"EDEKA BAUR, Gottlieber Str. 34, 78462 Konstanz",Mid-range,1000,1.0,2.93,...,10.0,3.23,10.0,1.25,0.600,3.88,0.40,2.91,NaN,NaN


In [3]:
#dropping the columns that are not required for the initial part of the assignment
data.drop(['Timestamp','Email Address','Your name','Grocery store name and street address (or URL if shopping online)','Average rental price near the grocery store'], axis = 1, inplace = True)

#naming the columns in a more convenient manner
data.rename(columns={"Country where you (and the grocery store) are": "Country", "Currency (e.g. EUR, GBP, USD)": "Currency","Price perception of the store brand":"Brand_Perception"}, inplace= True)

#previewing the data
data.head()

,Country,Currency,Brand_Perception,Apple 1 quantity (kg),Apple 1 price,Apple 2 quantity (kg),Apple 2 price,Apple 3 quantity (kg),Apple 3 price,Banana 1 quantity (kg),...,Eggs 2 quantity (count),Eggs 2 price,Eggs 3 quantity (count),Eggs 3 price,Chicken 1 quantity (kg),Chicken 1 price,Chicken 2 quantity (kg),Chicken 2 price,Chicken 3 quantity (kg),Chicken 3 price
0,Germany,EUR,Mid-range,1.0,2.93,2.0,3.22,1.0,1.75,1.0,...,6.0,1.75,4.0,1.99,0.312,3.97,0.600,3.87,0.400,2.90
1,Germany,EUR,Mid-range,1.0,1.95,1.0,2.93,0.8,2.93,1.0,...,6.0,2.73,10.0,1.63,0.394,5.02,0.430,7.17,0.311,9.12
2,Germany,EUR,Mid-range,1.0,2.93,1.0,1.25,2.0,3.42,1.0,...,10.0,3.23,10.0,1.25,0.600,3.88,0.400,2.91,NaN,NaN
3,Germany,EUR,Luxury (expensive),1.0,3.99,1.0,3.99,1.0,3.99,1.0,...,10.0,1.99,1.0,0.45,1.000,32.90,NaN,NaN,NaN,NaN
4,Germany,EUR,Budget (cheap),1.0,2.93,1.0,2.93,1.0,2.93,1.0,...,6.0,2.44,10.0,2.34,0.600,3.87,0.392,4.99,0.333,3.26


In [4]:
#converting all prices to a unified currency(USD) and unified quantity(unit)
for index in data.index:
    if data.iloc[index,1] == "EUR":
        for i in range(4,62,2):
            data.iloc[index,i] = (data.iloc[index,i]*1.187)/data.iloc[index,i-1]
    elif data.iloc[index,1] == "GBP":
        for k in range(4,62,2):
            data.iloc[index,k] = (data.iloc[index,k]*1.316)//data.iloc[index,i-1]
    elif data.iloc[index,1] == "MAD":
        for m in range(4,62,2):
            data.iloc[index,m] = (data.iloc[index,m]*0.109)/data.iloc[index,i-1]
    elif data.iloc[index,1] == "VND":
        for l in range(4,62,2):
            data.iloc[index,l] = (data.iloc[index,l]*0.000043)/data.iloc[index,i-1]
    elif data.iloc[index,1] == "KRW":
        for h in range(4,62,2):
            data.iloc[index,h] = (data.iloc[index,h]*0.00089)/data.iloc[index,i-1]
    elif data.iloc[index,1] == "GTQ":
        for g in range(4,62,2):
            data.iloc[index,g] = (data.iloc[index,g]*0.129)/data.iloc[index,i-1]
    elif data.iloc[index,1] == "USD":
        for g in range(4,62,2):
            data.iloc[index,g] = (data.iloc[index,g])/data.iloc[index,i-1]

data.head(5)

,Country,Currency,Brand_Perception,Apple 1 quantity (kg),Apple 1 price,Apple 2 quantity (kg),Apple 2 price,Apple 3 quantity (kg),Apple 3 price,Banana 1 quantity (kg),...,Eggs 2 quantity (count),Eggs 2 price,Eggs 3 quantity (count),Eggs 3 price,Chicken 1 quantity (kg),Chicken 1 price,Chicken 2 quantity (kg),Chicken 2 price,Chicken 3 quantity (kg),Chicken 3 price
0,Germany,EUR,Mid-range,1.0,3.47791,2.0,1.91107,1.0,2.077250,1.0,...,6.0,0.346208,4.0,0.590533,0.312,15.103814,0.600,7.656150,0.400,2.90
1,Germany,EUR,Mid-range,1.0,2.31465,1.0,3.47791,0.8,4.347387,1.0,...,6.0,0.540085,10.0,0.193481,0.394,15.123706,0.430,19.792535,0.311,9.12
2,Germany,EUR,Mid-range,1.0,3.47791,1.0,1.48375,2.0,2.029770,1.0,...,10.0,0.383401,10.0,0.148375,0.600,7.675933,0.400,8.635425,NaN,NaN
3,Germany,EUR,Luxury (expensive),1.0,4.73613,1.0,4.73613,1.0,4.736130,1.0,...,10.0,0.236213,1.0,0.534150,1.000,39.052300,NaN,NaN,NaN,NaN
4,Germany,EUR,Budget (cheap),1.0,3.47791,1.0,3.47791,1.0,3.477910,1.0,...,6.0,0.482713,10.0,0.277758,0.600,7.656150,0.392,15.110026,0.333,3.26


In [5]:
#Converting country to a categorical variable with codes to be later used in the Stan model
data.Country = pd.Categorical(data.Country)
data['Country_Coded'] = data.Country.cat.codes

In [6]:
#Converting Brand Perception to a categorical variable with codes to be later used in the Stan model
data.Brand_Perception = pd.Categorical(data.Brand_Perception)
data['Brand_Perception_Coded'] = data.Brand_Perception.cat.codes
data.head()

,Country,Currency,Brand_Perception,Apple 1 quantity (kg),Apple 1 price,Apple 2 quantity (kg),Apple 2 price,Apple 3 quantity (kg),Apple 3 price,Banana 1 quantity (kg),...,Eggs 3 quantity (count),Eggs 3 price,Chicken 1 quantity (kg),Chicken 1 price,Chicken 2 quantity (kg),Chicken 2 price,Chicken 3 quantity (kg),Chicken 3 price,Country_Coded,Brand_Perception_Coded
0,Germany,EUR,Mid-range,1.0,3.47791,2.0,1.91107,1.0,2.077250,1.0,...,4.0,0.590533,0.312,15.103814,0.600,7.656150,0.400,2.90,0,2
1,Germany,EUR,Mid-range,1.0,2.31465,1.0,3.47791,0.8,4.347387,1.0,...,10.0,0.193481,0.394,15.123706,0.430,19.792535,0.311,9.12,0,2
2,Germany,EUR,Mid-range,1.0,3.47791,1.0,1.48375,2.0,2.029770,1.0,...,10.0,0.148375,0.600,7.675933,0.400,8.635425,NaN,NaN,0,2
3,Germany,EUR,Luxury (expensive),1.0,4.73613,1.0,4.73613,1.0,4.736130,1.0,...,1.0,0.534150,1.000,39.052300,NaN,NaN,NaN,NaN,0,1
4,Germany,EUR,Budget (cheap),1.0,3.47791,1.0,3.47791,1.0,3.477910,1.0,...,10.0,0.277758,0.600,7.656150,0.392,15.110026,0.333,3.26,0,0


In [7]:
#creating data directroy for the products and the two multipliers
country_list = { 1: 'Germany', 2: 'Guatemala', 3: 'Morocco', 4: 'South Korea', 5: 'United Kingdon', 6:'United States', 7:'Vietnam'}

type_list = { 1:"Budget (cheap)",2:"Luxury (expensive)",3:"Mid-range"}

product_list = {1:"Apple",2:"Banana",3:"Tomatoes",4:"Potatos",5:"Flour",
             6:"Rice",7:"Milk",8:"Butter",9:"Eggs",10:"Chicken"}

In [8]:
#creating lists to feed later into the Stan Model
prices = []
countries = []
types = []
products = []

for i in range(1,11):
    product = product_list[i]
    price1 = data[[product+' 1 price','Country_Coded','Brand_Perception_Coded']].dropna()
    price2 = data[[product+' 2 price','Country_Coded','Brand_Perception_Coded']].dropna()
    price3 = data[[product+' 3 price','Country_Coded','Brand_Perception_Coded']].dropna()

    price_list = price1[product+' 1 price'].values.tolist()+ price2[product+' 2 price'].values.tolist() + price3[product+' 3 price'].values.tolist()
    country_list = price1['Country_Coded'].values.tolist()+ price2['Country_Coded'].values.tolist() + price3['Country_Coded'].values.tolist()
    type_list = price1['Brand_Perception_Coded'].values.tolist()+ price2['Brand_Perception_Coded'].values.tolist() + price3['Brand_Perception_Coded'].values.tolist()
    product_lister = len(product_list)*[i]

    prices += price_list
    countries += country_list
    types += type_list
    products += product_lister

In [9]:
#Defining the stan model
stan_code = '''

data {
    int<lower=1> N;    // number of prices collected
    int<lower=1> P;    // number of product categories for which data was collected
    int<lower=1> C;    // number of countries from which the data was collected
    int<lower=1> T;    // number of store types from which data was collected

    //data
    real<lower=0> prices[N];
    int countries[N];
    int types[N];
    int products[N];

    real<lower=0> alpha;   // fixed prior hyperparameter 
    real<lower=0> beta;    // fixed prior hyperparameter
}

parameters {
    real<lower=0> sigma;  // the standard deviation (the noise) we generate for our base price model
    real<lower=0> base_price[P]; //vector containing base prices for each of the product being modelled
    real<lower=0> country_multiplier[C]; //vector containing country multipliers 
    real<lower=0> store_multiplier[T]; //vector containing store perception multipliers 
}

model {
    sigma ~ gamma(alpha,beta); //generating standard deviation values for the prior

    for (i in 1:P){
        base_price[products[i]] ~ cauchy(1.5,2); //likelihood function for the base prices
    };

    for (i in 1:C){
        country_multiplier[countries[i]] ~ lognormal(1,2);
    };
    
    for (i in 1:T){
        store_multiplier[types[i]] ~ lognormal(1,2);
    };

    for (i in 1:N){
        prices[i] ~ normal(base_price[products[i]]*country_multiplier[countries[i]]*store_multiplier[types[i]], sigma); //likelihood
    };

}

'''
stan_model_1 = pystan.StanModel(model_code=stan_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_d4e955bf87ea80a87f1cffb2dfee1afd NOW.


In [ ]:
stan_data= {
    'prices': prices,
    'countries': countries,
    ''

}